In [163]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
df = pd.read_csv('HR-dataset.csv')

np.random.seed(42)
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

target = 'left'
features = df.columns.drop(target)
features = features.drop('empid')  # Удалим идентификатор пользователя как нерепрезентативный признак
print(features)

X, y = df[features].copy(), df[target]


Index(['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident',
       'promotion_last_5years', 'dept', 'salary'],
      dtype='object')


In [164]:
salary_ordinals = {'low': 1, 'medium': 2, 'high': 3}

X['dept'] = X['dept'].apply(X['dept'].value_counts().get)
X['salary'] = X['salary'].apply(salary_ordinals.get)
scaler = StandardScaler()
X = pd.DataFrame(data=scaler.fit_transform(X), columns=X.columns)


In [165]:
def estimate_accuracy(clf, X, y, cv=5):
    return cross_val_score(clf, X, y, cv=5, scoring='accuracy').mean()


In [166]:
tree = DecisionTreeClassifier(max_depth=30)
print("Decision tree:", estimate_accuracy(tree, X, y))


Decision tree: 0.9731310659108592


In [167]:
bagging_trees = BaggingClassifier(tree)
print("Decision tree bagging:", estimate_accuracy(bagging_trees, X, y))

Decision tree bagging: 0.9880660886962321


In [168]:
random_tree = DecisionTreeClassifier(max_features=int(np.sqrt(len(features))), max_depth=30)
print("Random tree:", estimate_accuracy(random_tree, X, y))


Random tree: 0.9778657330221184


In [169]:
bagging_random_trees = BaggingClassifier(random_tree)
print("Random tree bagging:", estimate_accuracy(bagging_random_trees, X, y))

Random tree bagging: 0.9902662443036567


In [170]:
random_forest = RandomForestClassifier(
    n_estimators=100,
    n_jobs=-1,
    max_features=int(np.sqrt(len(features))),
    max_depth=30)
print("Random Forest:", estimate_accuracy(random_forest, X, y))


Random Forest: 0.9920663109925532


In [171]:
random_forest = RandomForestClassifier(
    n_estimators=100,
    max_features=int(np.sqrt(len(features))),
    max_depth=30,
    oob_score=True,
    n_jobs=-1
)
random_forest.fit(X, y)
random_forest.oob_score_.mean()


0.9929995333022201

In [172]:
lr = LogisticRegression(solver='saga', max_iter=200)
lr.fit(X, y)
print("LR:", estimate_accuracy(lr, X, y))


LR: 0.7709770367900411


In [173]:
random_logreg = BaggingClassifier(
    lr,
    n_estimators=10,
    n_jobs=-1,
    random_state=42
)
print("Bagging for LR:", estimate_accuracy(random_logreg, X, y))


Bagging for LR: 0.7701104368122708


In [174]:
random_logreg = BaggingClassifier(
    lr,
    n_estimators=10,
    n_jobs=-1,
    max_features=0.5,
    random_state=42
)
print("Bagging for LR:", estimate_accuracy(random_logreg, X, y))


Bagging for LR: 0.7569754140268978


In [175]:
def plot_predictions(X, y, clf, proba=False, points_size=7, xlabel='x', ylabel='y'):
    """Fits the classifier on the data (X, y) and plots the result on a 2-D plane."""
    def get_grid(data):
        x_std, y_std = data.std(axis=0)
        x_min, x_max = data[:, 0].min() - x_std / 2, data[:, 0].max() + x_std / 2
        y_min, y_max = data[:, 1].min() - y_std / 2, data[:, 1].max() + y_std / 2
        return np.meshgrid(np.linspace(x_min, x_max, num=200),
                           np.linspace(y_min, y_max, num=200))
    
    clf.fit(X, y)
    xx, yy = get_grid(X)
    if proba:
        predicted = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1].reshape(xx.shape)
    else:
        predicted = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
        
    plt.figure(figsize=(10.0, 10.0))
    plt.pcolormesh(xx, yy, predicted, cmap=plt.cm.coolwarm, alpha=0.1)
    plt.scatter(X[:, 0], X[:, 1], c=y, s=points_size, cmap=plt.cm.coolwarm, alpha=0.90)
    plt.ylim([yy.min(),yy.max()])
    plt.xlim([xx.min(),xx.max()])
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    
    return clf


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
df = pd.read_csv('HR-dataset.csv')

np.random.seed(42)
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

target = 'left'
features = df.columns.drop(target)
features = features.drop('empid')  # Удалим идентификатор пользователя как нерепрезентативный признак

# Кодируем категориальные признаки
label_encoder = LabelEncoder()
df['dept'] = label_encoder.fit_transform(df['dept'])
salary_ordinals = {'low': 1, 'medium': 2, 'high': 3}
df['salary'] = df['salary'].apply(salary_ordinals.get)

X, y = df[features].copy(), df[target]

scaler = StandardScaler()
X = pd.DataFrame(data=scaler.fit_transform(X), columns=X.columns)

def estimate_accuracy(clf, X, y, cv=5):
    return cross_val_score(clf, X, y, cv=5, scoring='accuracy').mean()

tree = DecisionTreeClassifier(max_depth=30)
print("1:", estimate_accuracy(tree, X, y))


1: 0.9739315994220294


In [2]:
from sklearn.ensemble import BaggingClassifier


bagging_trees = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100, random_state=42)
print("2:", estimate_accuracy(bagging_trees, X, y))


2: 0.9899328442814271


In [3]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from math import sqrt

# Определяем количество признаков
num_features = X.shape[1]
# Вычисляем квадратный корень из числа признаков
sqrt_num_features = int(sqrt(num_features))

# Создаем базовый классификатор DecisionTreeClassifier
base_tree = DecisionTreeClassifier()

# Создаем BaggingClassifier с базовым классификатором DecisionTreeClassifier
# и устанавливаем параметр max_features='sqrt' для выбора sqrt(d) случайных признаков
bagging_classifier = BaggingClassifier(base_estimator=base_tree, 
                                       n_estimators=100,
                                       max_features=sqrt_num_features)

# Оцениваем качество классификации с помощью кросс-валидации
bagging_accuracy = estimate_accuracy(bagging_classifier, X, y)
print("3:", bagging_accuracy)


3: 0.9667321329331999


In [4]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from math import sqrt

# Определяем количество признаков
num_features = X.shape[1]
# Вычисляем квадратный корень из числа признаков
sqrt_num_features = int(sqrt(num_features))

# Создаем базовый классификатор DecisionTreeClassifier с установленным параметром max_features='sqrt'
base_tree = DecisionTreeClassifier(max_features='sqrt')

# Создаем BaggingClassifier с базовым классификатором DecisionTreeClassifier
bagging_classifier = BaggingClassifier(base_estimator=base_tree, 
                                       n_estimators=100)

# Оцениваем качество классификации с помощью кросс-валидации
bagging_accuracy = estimate_accuracy(bagging_classifier, X, y)
print("4:", bagging_accuracy)


4: 0.9918662665332889


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

df = pd.read_csv('HR-dataset.csv')

target = 'left'
features = df.columns.drop(target).drop('empid')

label_encoder = LabelEncoder()
df['dept'] = label_encoder.fit_transform(df['dept'])
salary_ordinals = {'low': 1, 'medium': 2, 'high': 3}
df['salary'] = df['salary'].apply(salary_ordinals.get)

X, y = df[features], df[target]

# Создание RandomForestClassifier с параметрами по умолчанию
rf_clf = RandomForestClassifier(random_state=42)
# Оценка качества с использованием cross_val_score
rf_scores = cross_val_score(rf_clf, X, y, cv=10, n_jobs=-1)
print("Среднее значение оценки качества для RandomForestClassifier:", rf_scores.mean())

# Исследование зависимости от количества деревьев
n_estimators_values = [10, 50, 100, 200, 300]
scores_n_estimators = []

for n_estimators in n_estimators_values:
    rf_clf_n_estimators = RandomForestClassifier(n_estimators=n_estimators, random_state=42)
    scores = cross_val_score(rf_clf_n_estimators, X, y, cv=10, n_jobs=-1)
    scores_n_estimators.append(scores.mean())

# Исследование зависимости от количества признаков
max_features_values = [1, int(np.sqrt(X.shape[1])), int(X.shape[1] / 3), X.shape[1]]
scores_max_features = []

for max_features in max_features_values:
    rf_clf_max_features = RandomForestClassifier(max_features=max_features, random_state=42)
    scores = cross_val_score(rf_clf_max_features, X, y, cv=10, n_jobs=-1)
    scores_max_features.append(scores.mean())

# Исследование зависимости от глубины дерева
max_depth_values = [None, 5, 10, 15, 20]
scores_max_depth = []

for max_depth in max_depth_values:
    rf_clf_max_depth = RandomForestClassifier(max_depth=max_depth, random_state=42)
    scores = cross_val_score(rf_clf_max_depth, X, y, cv=10, n_jobs=-1)
    scores_max_depth.append(scores.mean())

# Построение графиков
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot(n_estimators_values, scores_n_estimators, marker='o')
plt.title('Зависимость от количества деревьев')
plt.xlabel('Количество деревьев')
plt.ylabel('Среднее значение оценки качества')

plt.subplot(1, 3, 2)
plt.plot(max_features_values, scores_max_features, marker='o')
plt.title('Зависимость от количества признаков')
plt.xlabel('Количество признаков')
plt.ylabel('Среднее значение оценки качества')

plt.subplot(1, 3, 3)
plt.plot(max_depth_values, scores_max_depth, marker='o')
plt.title('Зависимость от глубины дерева')
plt.xlabel('Глубина дерева')
plt.ylabel('Среднее значение оценки качества')

plt.tight_layout()
plt.show()
